In [1]:
!pip install -qU langchain-google-genai


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [76]:
import getpass
import os
from dotenv import load_dotenv
import base64
from langchain_core.prompts import HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

In [77]:
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")


In [124]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    
)

In [125]:
chat_prompt = ChatPromptTemplate.from_messages(
    messages=[
        SystemMessage(
            content='''You are well experienced dietician and nutritionist. You will be given the Gae, height, weight and dietary precautions and goals of the person. 
                        You will be given a product and its ingredients select one from each category based on the ingredients and health goals of the person.
                        Category 1: how frequently can i consume it based on my weight goals:
                        1. daily consumption
                        2. weekly consumption
                        3. monthly consumption
                        Category 2: and as per my diet is this food :
                        1. nutritional
                        2. recreations
                        3.  or regular consumption for me

                        Category 3: If the food product follows any of the following:
                        1. Organic
                        2. Sustainable: 
                        3. Supports Small Businesses
                        4. Animal Cruelty-Free: 

                        Category 4: flag alergens if they are related to my allergy otherwise don't flag a warning
                        Category 5: you need to also flag diet type match this with the input of user info and flag if it is suitable for their dietary type or restrictions.
                        Category 6: Alert if there is a higher presence of nutrients desired in low qty (fats, sugar, sodium, calories)
                        Your final answer should only contain tags among one option of each category based on product nutrition analysis.
                        If the answer is None just drom that category output.
                        Finally if the 4 out of 6 categories are suitable for this user add that as highly recommended product.
                        Your next input will be the info of the person, the name of the product, and screeshot of the ingredients
            '''),
        HumanMessagePromptTemplate.from_template(
            template='''
            The persons info is as follow:
            Person info: I am a {gender} who is {age} years old my weight is {weight}, my heaight is {height},i am {diet_type} and i want to {health_goal}. Allergic to {allergen}.
            Product info: {product_info}
            'image_url': "data:image/jpeg;base64,{image_data}"
           
            
            '''),
       
    ]
)

In [126]:
chat_prompt

ChatPromptTemplate(input_variables=['age', 'allergen', 'diet_type', 'gender', 'health_goal', 'height', 'image_data', 'weight'], input_types={}, partial_variables={}, messages=[SystemMessage(content="You are well experienced dietician and nutritionist. You will be given the Gae, height, weight and dietary precautions and goals of the person. \n                        You will be given a product and its ingredients select one from each category based on the ingredients and health goals of the person.\n                        Category 1: how frequently can i consume it based on my weight goals:\n                        1. daily consumption\n                        2. weekly consumption\n                        3. monthly consumption\n                        Category 2: and as per my diet is this food :\n                        1. nutritional\n                        2. recreations\n                        3.  or regular consumption for me\n\n                        Category 3: If the food

In [128]:
chain = chat_prompt | model 

# Path to the image file
image_path = r"C:\Users\DELL\Downloads\13-Google-GenAI-Hack-24\GenAI-Google--Hack-24\RAG\Pringles.png"

# Read and convert the image to base64
with open(image_path, "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode("utf-8")




In [129]:
response_from_model = chain.invoke(
    {
        
        "image_data": image_data,
        "age":"34",
        "weight": "168kg",
        "gender": "male",
        "height":"5 ft",
        "diet_type": "jain",
        "health_goal": "loose 50 kgs",
        "allergen": "peanuts"
        "product_info":"Pringles Potato Chips Desi Masala Tadka Flavor Pack of 4, Crispy Snack for Game Nights Chips (4 x 102 g)"
        

    }
)

In [130]:
response_from_model.content


'2. Weekly consumption\n2. Recreational\nNone\n4. Contains peanuts \n5. Not suitable for Jain diet\n6. Higher presence of fats, sugar, sodium, calories \n'